In [1]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('data/mturk_results_0-4002.csv').rename(index=str, columns =
            {'Input.text':'Text', 'Answer.Reaction.on':'Reaction', 
             'Answer.Report.on':'Report', 'Answer.Request.on':'Request',
             'Answer.Urgent.on':'Urgency', 'Answer.NotRelevant.on':'NotRelevant'})
df.pop('Answer.NotUrgent.on')
df['Urgency'] = df['Urgency'].map(lambda x: 1 if x >= 2 else 0)

print(df.shape)
df.head()

(4003, 6)


,Text,Reaction,Report,Request,NotRelevant,Urgency
0,\n#Harvey a marathon not a sprint | Severe th...,0,1,0,2,0
1,"your rescue boats, vehicles, volunteer craft ...",0,0,1,2,0
2,! @Billcassidy on adding short-term debt limit...,1,1,0,1,0
3,"!! According to #PBS, #Houston convention cen...",0,0,3,0,0
4,"""#FEMA flood maps outdated do not reflect incr...",2,0,0,1,0


In [75]:
#now loop through the rows and only keep those whose ties can be broken
relevancy = []
categories = ['NotRelevant', 'Request', 'Report', 'Reaction']
for row in df.itertuples(index=False):
    majority = -1
    for c in categories:
        if getattr(row, c) >= 2:
            majority = categories.index(c)
            break
    relevancy.append(majority)

Run the following code if you want to drop all the rows without a majority vote.

In [68]:
#now drop all the rows that don't have a majority
df['Relevancy'] = relevancy
df = df[df.Relevancy > -1]
df.drop(categories, axis=1, inplace=True)
print(df.shape)
df.head()

(3510, 3)


,Text,Urgency,Relevancy
0,\n#Harvey a marathon not a sprint | Severe th...,0,0
1,"your rescue boats, vehicles, volunteer craft ...",0,0
3,"!! According to #PBS, #Houston convention cen...",0,1
4,"""#FEMA flood maps outdated do not reflect incr...",0,3
5,"""100,000 homes have been damaged by #Harvey an...",0,2


In [69]:
df.to_csv('data/mturk_results_0-4000_processed.csv', index=False)

Run this code if you want to keep all the tweets but indicate which ones do not have a majority vote for relevancy.

In [76]:
#now drop all the rows that don't have a majority
df['Relevancy'] = relevancy
df.drop(categories, axis=1, inplace=True)
df.head()

,Text,Urgency,Relevancy
0,\n#Harvey a marathon not a sprint | Severe th...,0,0
1,"your rescue boats, vehicles, volunteer craft ...",0,0
2,! @Billcassidy on adding short-term debt limit...,0,-1
3,"!! According to #PBS, #Houston convention cen...",0,1
4,"""#FEMA flood maps outdated do not reflect incr...",0,3


In [77]:
df.to_csv('data/mturk_results_0-4000_manual.csv', index=False)

In [22]:
df = pd.read_csv('data/labeled_prelim.csv').dropna().astype(
        {'Relevancy':np.int32, 'Urgency':np.int32}).reset_index(drop=True)
df.pop('Id')

df2 = pd.read_csv('data/mturk_0-4000_manual4.csv')
df2 = df2[df2.Relevancy > -1]

df = pd.concat([df, df2], ignore_index=True)
print(df.shape)
df.head()

(5155, 3)


,Text,Relevancy,Urgency
0,More millions in #Afghanistan even with ZERO a...,0,0
1,These are the last post my brother made on soc...,2,1
2,In @cityofcc listening to local officials abou...,0,0
3,So so so damn proud of @5ugarcane who is tirel...,3,0
4,How can you help with #Harvey disaster respons...,0,0


In [23]:
sum(map(lambda x: 1 if x > 0 else 0, df['Relevancy']))

2039

In [35]:
df_pos = df[df.Relevancy > 0]
df_neg = df[df.Relevancy == 0]
df_neg = df_neg.reindex(np.random.permutation(df_neg.index))[:2039]

In [45]:
df_final = pd.concat([df_neg, df_pos])
df_final = df_final.reindex(np.random.permutation(df_final.index))
df_final = df_final.reset_index(drop=True)
df_final.head()

,Text,Relevancy,Urgency
0,That #HarveyStorm water is waist deep. My resp...,0,0
1,Find out how you help those affected by Hurric...,1,0
2,#MountainView heroes deployed to help with #Ha...,3,0
3,"To help those impacted by #HurricaneHarvey, we...",0,0
4,So much flooding in Houston. Wow! Just tuning ...,0,0


In [46]:
df_final.to_csv('data/final_dataset.csv', index=False)